# Introducción

## El objetivo del presente trabajo práctico es aplicar los conceptos teóricos y práticos aprendidos de la materia a la predicción de una feature en el dataset de lluvia de hamburguesas y a su vez relatar en el notebook el análisis y procedimiento utilizado para poder generar dicha predicción.

In [1]:
import pandas as pd
import numpy as np
import requests
from matplotlib import pyplot as plt
import seaborn as sns
from sklearn.metrics import accuracy_score
from pandas_profiling import ProfileReport

In [ ]:
df_1 = pd.read_csv(
    'https://docs.google.com/spreadsheets/d/1gvZ03uAL6THwd04Y98GtIj6SeAHiKyQY5UisuuyFSUs/export?format=csv', low_memory=False
)
df_2 = pd.read_csv(
    'https://docs.google.com/spreadsheets/d/1wduqo5WyYmCpaGnE81sLNGU0VSodIekMfpmEwU0fGqs/export?format=csv', low_memory=False
)
pd.options.display.max_columns = None

df = df_2.merge(df_1, left_on = 'id', right_on = 'id')
df.sort_values(by=['id'], inplace=True, ascending=True)

df_original = df.copy()

df.sample(10)

# Feature engineering
### Buscamos filas duplicadas
### Consideramos a una fila duplicada unicamente cuando se repite el ID de la medicion

In [ ]:
df[df.duplicated(subset=['id'],keep=False)]

### Vemos que no hay duplicados, pero verifiquemos además que el tipo para cada feature es razonable:

In [ ]:
df.dtypes

#### Sabiendo que el pandas trata los strings como 'object', inmediatamente notamos que hay una feature que debería estar marcada como float pero se está tratando como un object: presion_atmosferica_tarde. Por dicho motivo procedemos a transformarla a numérica:

Si hacemos directamente to_numeric(), vemos que encuentra un valor que no puede convertir por tener dos simbolos '.' de decimal, por lo que asumimos que fue un error
al cargar el dato de presion_atmosferica_tarde, y preferimos dejar ese casillero en NaN que eliminar el registro completo:

In [ ]:
df['presion_atmosferica_tarde'].replace('.+\..+\..+', np.nan, inplace=True, regex=True)
df['presion_atmosferica_tarde'] = pd.to_numeric(df['presion_atmosferica_tarde'])

## Variables __categoricas__
1. Barrio 
2. direccion_viento_tarde
3. direccion_viento_temprano
4. rafaga_viento_max_direccion
5. llovieron_hamburguesas_hoy

### Comenzamos por reemplazar los valores nulos en direcciones de viento y ráfagas por una nueva opcion que puede tomar la feature : "Otra direccion"

In [ ]:
df['direccion_viento_tarde'].replace(np.nan, 'Otra direccion', inplace=True)
df['direccion_viento_temprano'].replace(np.nan, 'Otra direccion', inplace=True)
df['rafaga_viento_max_direccion'].replace(np.nan, 'Otra direccion',inplace=True)

### Luego verificamos que ninguna feature categorica tenga valores repetidos o sin sentido por errores de tipado 

In [ ]:
df.barrio.value_counts().to_frame()

In [ ]:
df.direccion_viento_temprano.value_counts().to_frame()

In [ ]:
df.direccion_viento_tarde.value_counts().to_frame()

In [ ]:
df.rafaga_viento_max_direccion.value_counts().to_frame()

In [ ]:
df.llovieron_hamburguesas_hoy.value_counts().to_frame()

#### Vemos que todos los valores son unicos, no hay necesidad de alterar las features y podemos seguir con la conversion a valores numericos

## Variables __numéricas__


### Comenzamos por buscar cuáles columnas tienen missings

In [ ]:
df.fillna(np.nan, inplace = True)
df_nulos = pd.DataFrame({'Cantidad de nulos': df.isnull().sum(), 'Porcentaje de nulos': (df.isnull().mean() * 100)})
df_nulos.sort_values(by=['Porcentaje de nulos'])

Vemos que algunos de los features tienen menos de un 1.5% de NaNs, por lo cual usar la media para rellenarlos no implicaría una disonancia significativa en la información, sinó más bien un suavizado de esos casos vacíos.

In [ ]:
df['temp_max'].replace(np.nan, df['temp_max'].mean(), inplace = True)
df['temp_min'].replace(np.nan, df['temp_min'].mean(), inplace = True)
df['temperatura_temprano'].replace(np.nan, df['temperatura_temprano'].mean(), inplace = True)
df['velocidad_viendo_temprano'].replace(np.nan, df['velocidad_viendo_temprano'].mean(), inplace = True)

df_nulos = pd.DataFrame({'Cantidad de nulos': df.isnull().sum(), 'Porcentaje de nulos': (df.isnull().mean() * 100)})
df_nulos.sort_values(by=['Porcentaje de nulos'])

Para las features con alta cardinalidad de nulos (horas_de_sol, mm_evaporados_agua, nubosidad_temprano, nubosidad_tarde) analizaremos cada caso en particular para observar la relacion de dichos nulos con la variable target. 
Desconocemos como se computa la nubosidad, y por eso no podemos afirmar que se trata de columnas con missings at random (MAR). Es decir, no podemos afirmar que la falta de informacion en nubosidad dependa de horas_de_sol, (lo mismo aplica para las nubosidades y los milimetros evaporados de agua), por ende trataremos estos missings como MCAR completely at random.

# Formulación de Preguntas y Visualización
## Planteamos un heatmap donde se pueden ver las correlaciones entre todas las variables númericas y la feature target, y en base a dicho gráfico y al dataset, planteamos una serie de preguntas que guiaran el análisis

In [ ]:
df_numerico = df.drop(['barrio','dia','direccion_viento_tarde','direccion_viento_temprano','id','llovieron_hamburguesas_hoy','rafaga_viento_max_direccion'], axis=1)
df_numerico['llovieron_hamburguesas_al_dia_siguiente'] = df['llovieron_hamburguesas_al_dia_siguiente'].map({'si': 1, 'no': 0})

fig, ax = plt.subplots(figsize=(15, 15))

hm = sns.heatmap(df_numerico.corr(),annot=True)

plt.show()

 1. ¿Suponiendo desconocimiento de todas las features excepto el target, que tan probable es que lluevan hamburguesas al dia siguiente?
 2. ¿De qué manera se relaciona el barrio con que lluevan hamburguesas al dia siguiente?
 3. ¿Si llovieron hamburguesas hoy, qué tan probable es que lluevan hamburguesas mañana?
 4. ¿Cómo afecta la feature mm_lluvia_dia al target? ¿Y qué sucede con los mm evaporados?
 5. ¿Hay correlación entre la feature horas_de_sol y la lluvia de hamburguesas del dia siguiente?
 6. ¿Cómo se relacionan la nubosidad_temprano y la nubosidad_tarde con el target?
 7. ¿Las direcciones del viento temprano y tarde, estarán involucradas en la búsqueda del baseline?
 8. ¿Cómo afectan las velocidades de los vientos tempranos y tardes al baseline?
 9. ¿En cuánto al viento, las ráfagas tienen efecto en el target?
 10. ¿Los máximos y mínimos de temperatura, indican algo sobre si llueven o no hamburguesas al dia siguiente?
 11. ¿Podemos encontrar alguna correlación entre la máxima temperatura en el día, y los milímetros que se evaporan de lluvia?
 12. ¿Tiene relación la humedad con la variable target?
 13. ¿Encontraremos alguna correlacion entre humedad, temperatura y el target, tal cual indicaría el sentido común?
 14. ¿Es observable una correlación entre la presión atmosférica y el target?
 15. ¿Qué correlaciones encontramos entre horas_de_sol, nubosidad_tarde y el target?
 16. ¿Cómo se comporta el target cuando las features que prometen mejor predicción están todas en valores nulos?
 17. ¿Cómo afecta al target la cantidad de features nulas en una medición?
 18. ¿Son de utilidad las mediciones donde el target tiene el valor NaN?
 
Buscamos entender cada uno de los datos, cómo se relacionan entre sí y ver que conclusiones podemos obtener. A partir de ellos vamos a determinar los factores que determinan si lloverán hamburguesas al dia siguiente, y verificaremos cuáles features son reelevantes de mantener en el dataset.

### 1.¿Suponiendo desconocimiento de todas las features excepto el target, que tan probable es que lluevan hamburguesas al dia siguiente?

In [ ]:
plt.figure(figsize=(5.4, 3.8), dpi=150)
labels = ['No', 'Si']
sizes = df['llovieron_hamburguesas_al_dia_siguiente'].value_counts()
explode = (0, 0.1)
plt.title('Llueven hamburguesas al dia siguiente')
plt.pie(sizes, explode=explode, data = df['llovieron_hamburguesas_al_dia_siguiente'],labels = labels ,autopct='%1.1f%%',
        shadow=True, startangle=90)
plt.show()

Podemos corroborar, que desconociendo el resto de features, para el 77.6% de los registros en el dataset al dia siguiente no llueven hamburguesas. Esto quiere decir que si armaramos una baseline que simplemente devuelva siempre "NO", el accuracy ya sería de ese porcentaje. Sin embargo, no solo es un accuracy que no nos satisface, sinó que resulta en algo poco reutilizable que desperdicia el resto de información.

### 2. ¿De qué manera se relaciona el barrio con que lluevan hamburguesas al dia siguiente?

In [ ]:
lluvurguesas_por_barrio = df.groupby(by="barrio")['llovieron_hamburguesas_al_dia_siguiente'].value_counts(normalize=True)
lluvurguesas_por_barrio = lluvurguesas_por_barrio.to_frame()
lluvurguesas_por_barrio.rename(columns={'llovieron_hamburguesas_al_dia_siguiente':'porcentaje_por_barrio'},inplace=True)
lluvurguesas_por_barrio.reset_index(inplace=True)

g = sns.catplot(
    data=lluvurguesas_por_barrio, kind="bar",
    x="barrio", y="porcentaje_por_barrio", hue="llovieron_hamburguesas_al_dia_siguiente", order=df["barrio"].value_counts().index,
    ci=None, height=8,aspect=2
)
plt.title("Porcentaje de lluvia de hamburguesas según barrio", fontsize=25)
g.despine(left=True)
plt.ylim(0,1)
plt.xticks(rotation=90)
g.set_axis_labels("Barrio", "Porcentaje de lluvia de hamburguesas",  fontsize=15)
plt.show()

Podemos ver que en ningún barrio es predominante el "SI" para la feature llovieron_hamburguesas_al_dia_siguiente, por lo que no podemos hacer ninguna afirmación positiva para la lluvia de hamburguesas según el barrio. Sin embargo, podemos notar que ciertos barrios como Parque Patricios y Villa Pueryrredón, tienen lluvia de hamburguesas al dia siguiente más seguido de lo normal (un \~30% de las veces), mientras que barrios como Villa Crespo y Villa Santa Rita rondan números mas bajos (\~10%).

### 3.¿Si llovieron hamburguesas hoy, qué tan probable es que lluevan hamburguesas mañana?

In [ ]:
fig, axes = plt.subplots(nrows=1, ncols=2, figsize=[6.4 * 2, 4.8], dpi=100)

sns.countplot(data=df, x='llovieron_hamburguesas_hoy', hue='llovieron_hamburguesas_al_dia_siguiente', ax=axes[0])
axes[0].set_title("Cantidad de veces que llovieron hamburguesas \n" +  "al dia siguiente según si llovieron hoy")

sns.barplot(
    data=df.groupby("llovieron_hamburguesas_hoy")
    .llovieron_hamburguesas_al_dia_siguiente.value_counts(normalize=True)
    .rename("llovieron_al_dia_siguiente_prop")
    .reset_index(),
    x='llovieron_hamburguesas_hoy',
    y="llovieron_al_dia_siguiente_prop",
    hue='llovieron_hamburguesas_al_dia_siguiente',
    ax=axes[1],
)
axes[1].set_ylabel("% intragrupo")
axes[1].set_title("Porcentaje para lluvia de hamburguesas\n" +  "al dia siguiente según si llovieron hoy")

plt.show()

Podemos ver que si bien la cantidad de veces que llueven hamburguesas al dia siguiente es prácticamente la misma en ambas mitades del primer gráfico, cuando lo analizamos en porcentajes, vemos que si llovio el dia de hoy, la probabilidad de que lluevan hamburguesas al dia siguiente aumenta radicalmente. Concluimos de esta visualización, que si llovieron hamburguesas hoy, hay practicamente un 45% de probabilidades que llueva mañana, mientras que si hoy no llovieron hamburguesas, entonces esa probabilidad es mucho menor, de aproximadamente el 15%

### 4.¿Cómo afecta la feature mm_lluvia_dia al target? ¿Y qué sucede con los mm evaporados?

In [ ]:
fig = plt.figure(figsize=[6.4 * 2, 4.8], dpi=100)

df_llovio_h_maniana = df[df.llovieron_hamburguesas_al_dia_siguiente == 'si']
df_no_llovio_h_maniana = df[df.llovieron_hamburguesas_al_dia_siguiente == 'no']

plt.hist(df_no_llovio_h_maniana.mm_lluvia_dia, alpha=1, range=[0, 5], label='No llovieron hamburguesas al dia siguiente')
plt.hist(df_llovio_h_maniana.mm_lluvia_dia, alpha=0.75, range=[0, 5], label='Llovieron hamburguesas al dia siguiente')
plt.legend()
plt.title('mm lluvia - llovieron hamburguesas al dia siguiente')
plt.ylabel("Cantidad de mediciones")
plt.xlabel("mm de lluvia")
plt.yscale('log')
plt.show()


Se puede observar que la distribución de los mm de lluvia es muy similar para ámbos valores del target, por lo que no nos llevaría a ninguna estimación concluyente del target. Esto se puede notar ya que si bien el número de mediciones para cuando el target es positivo es mucho menor que cuando es negativo, se mantiene la proporción del dataset de aproximadamente 80% de casos negativos vs 20% positivos, y dentro de dicha proporcion, ambas distribuciones acumulan la mayor cantidad de mediciones en valores entre 0 y 1,5 mm de lluvia. 
Veamos si para los mm evaporados de agua en el día obtenemos una distribución mejor indicadora del target:

In [ ]:
fig = plt.figure(figsize=[6.4 * 2, 4.8], dpi=100)

df_llovio_h_maniana = df[df.llovieron_hamburguesas_al_dia_siguiente == 'si']
df_no_llovio_h_maniana = df[df.llovieron_hamburguesas_al_dia_siguiente == 'no']

plt.hist(df_no_llovio_h_maniana.mm_evaporados_agua, alpha=1, range=[0, 30], label='No llovieron hamburguesas al dia siguiente')
plt.hist(df_llovio_h_maniana.mm_evaporados_agua, alpha=0.75, range=[0, 30], label='Llovieron hamburguesas al dia siguiente')
plt.title('mm evaporados - llovieron hamburguesas al dia siguiente')
plt.ylabel("Cantidad de mediciones")
plt.xlabel("mm evaporados")
plt.yscale('log')
plt.legend()
plt.show()

Nuevamente observamos que las distribuciones en la visualización mantienen la proporción de 80% de casos de target negativo, y dentro de dicha proporción ambos casos acumulan la mayor cantidad de mediciones entre 0mm y 15mm por lo que a priori los milímetros evaporados no parecen ser un gran indicador para predecir el mismo. Para considerar una relación con la variable target deberiamos apreciar diferencias más significativas.

### 5.¿Hay correlación entre la feature horas_de_sol y la lluvia de hamburguesas del dia siguiente?


In [ ]:
fig = plt.figure(figsize=[6.4 * 2, 4.8], dpi=100)

df_llovio_h_maniana = df[df.llovieron_hamburguesas_al_dia_siguiente == 'si']
df_no_llovio_h_maniana = df[df.llovieron_hamburguesas_al_dia_siguiente == 'no']

plt.hist(df_no_llovio_h_maniana.horas_de_sol, range=[0,15], label='No llovieron hamburguesas al dia siguiente')
plt.hist(df_llovio_h_maniana.horas_de_sol, alpha=0.75, range=[0,15], label='Llovieron hamburguesas al dia siguiente')
plt.title('Horas de sol - llovieron hamburguesas al dia siguiente')
plt.ylabel("Cantidad de mediciones")
plt.xlabel("Horas de sol")
plt.legend()
plt.show()

Comparando ámbas distribuciones, vemos grandes diferencias entre las horas de sol según el valor del target. Cuando al día siguiente llueven hamburguesas, el gráfico naranja nos indica que suele haber en general pocas horas de sol. Caso contrario, en el gráfico azul vemo que la mayoría de mediciones en los cuales no llueven hamburguesas al día siguiente se agrupan a partir de aproximadamente 8 horas de sol.

Sin embargo, debemos analizar que sucede cuando horas de sol tiene missings. Dado que es una feature con una cardinalidad muy grande de valores nulos, amerita un análisis en detalle.

In [ ]:
fig =  plt.figure(figsize=(5.4, 3.8), dpi=150)\

df_horas_sol_desconocidas = df[(df['horas_de_sol'].isna())]

total_dias_lluvia = sum(df_horas_sol_desconocidas.llovieron_hamburguesas_al_dia_siguiente == 'si')
total_dias_sin_lluvia = sum(df_horas_sol_desconocidas.llovieron_hamburguesas_al_dia_siguiente == 'no')

sns.countplot(data=df_horas_sol_desconocidas, x='llovieron_hamburguesas_al_dia_siguiente', hue='llovieron_hamburguesas_al_dia_siguiente')
plt.title("Cantidad de mediciones en las que llovieron hamburguesas al día \n siguiente para horas de sol no conocidas")
plt.xlabel("Llovieron hamburguesas al día siguiente")
plt.axhline(y=total_dias_sin_lluvia, color="C0", label=f"{total_dias_sin_lluvia} dias", linestyle='--')
plt.axhline(y=total_dias_lluvia, color="C1", label=f"{total_dias_lluvia} dias",  linestyle='--')
plt.legend(loc='best')
plt.show()

Del gráfico de barras vemos que aproximadamente el 80% de los casos en los que no conocemos las horas de sol, tampoco llovieron hamburguesas al dia siguiente, manteniendo asi la propoción del dataset original mencionada con anterioridad. La falta de esta feature no genera anomal

### 6. ¿Cómo se relacionan la nubosidad_temprano y la nubosidad_tarde con el target?



In [ ]:
fig = plt.figure(figsize=[6.4 * 2, 4.8], dpi=100)

df_llovio_h_maniana = df[df.llovieron_hamburguesas_al_dia_siguiente == 'si']
df_no_llovio_h_maniana = df[df.llovieron_hamburguesas_al_dia_siguiente == 'no']
rango_nubosidad_temp = [0,9]

plt.hist(df_no_llovio_h_maniana.nubosidad_temprano, range=rango_nubosidad_temp, label='No llovieron hamburguesas al dia siguiente')
plt.hist(df_llovio_h_maniana.nubosidad_temprano, alpha=0.75, range=rango_nubosidad_temp, label='Llovieron hamburguesas al dia siguiente')
plt.title('nubosidad temprano - llovieron hamburguesas al dia siguiente')
plt.ylabel("Cantidad de mediciones")
plt.xlabel("Nubosidad temprano")
plt.legend()
plt.show()


Vemos que la distribución en ambos histogramas es muy diferente, pues la forma nos indica que es mucho mas probable que lluevan hamburguesas al día siguiente si la nubosidad es alta en comparación a cuando es baja. Además, cuando llovieron hamburguesas al día siguiente, la nubosidad muestra una tendencia siempre creciente, como una proporcionalidad directa, mientras que en el caso contrario, se observa una forma parabólica, es decir, hay muchas mediciones que se agrupan entre los rangos de valores 0-2 y 6-8 de nubosidad.

In [ ]:
fig = plt.figure(figsize=[6.4 * 2, 4.8], dpi=100)

df_llovio_h_maniana = df[df.llovieron_hamburguesas_al_dia_siguiente == 'si']
df_no_llovio_h_maniana = df[df.llovieron_hamburguesas_al_dia_siguiente == 'no']
rango_nubosidad_tarde = [0,9]

plt.hist(df_no_llovio_h_maniana.nubosidad_tarde, range=rango_nubosidad_tarde, label='No llovieron hamburguesas al dia siguiente')
plt.hist(df_llovio_h_maniana.nubosidad_tarde,alpha=0.75, range=rango_nubosidad_tarde, label='Llovieron hamburguesas al dia siguiente')
plt.title('Nubosidad - llovieron hamburguesas al dia siguiente')
plt.ylabel("Cantidad de mediciones")
plt.xlabel("Nubosidad tarde")
plt.legend()
plt.show()

Para la nubosidad a la tarde, la visualización resulta muy similar al caso de la nubosidad temprana, tanto para cuando llueven hamburguesas al dia siguiente como cuando no lo hace. Ambos graficos nos indican que la mayoría de casos donde han llovido hamburguesas al dia siguiente se agrupan para nubosidades entre 5 y 8. Creemos que debido a esta tan marcada diferencia, esta feature será de suma utilidad cuando haya que predecir el target. 

In [ ]:
fig, axes = plt.subplots(nrows=1, ncols=2, figsize=(11,3.8), dpi=125)

df_nubosidad_temprana_desconocida = df[(df['nubosidad_temprano'].isna())]
total_dias_lluvia_nubosidad_temprana_desc = sum(df_nubosidad_temprana_desconocida.llovieron_hamburguesas_al_dia_siguiente == 'si')
total_dias_sin_lluvia_nubosidad_temprana_desc = sum(df_nubosidad_temprana_desconocida.llovieron_hamburguesas_al_dia_siguiente == 'no')

sns.countplot(data=df_nubosidad_temprana_desconocida, x='llovieron_hamburguesas_al_dia_siguiente', hue='llovieron_hamburguesas_al_dia_siguiente', ax=axes[0])
axes[0].set_title("Cantidad de dias que llovieron hamburguesas al día \n siguiente para nubosidad temprano desconocida")
axes[0].set_xlabel("Llovieron hamburguesas al día siguiente")
axes[0].axhline(y=total_dias_lluvia_nubosidad_temprana_desc, color="C1", label=f"{total_dias_lluvia_nubosidad_temprana_desc} dias",  linestyle='--')
axes[0].axhline(y=total_dias_sin_lluvia_nubosidad_temprana_desc, color="C0", label=f"{total_dias_sin_lluvia_nubosidad_temprana_desc} dias", linestyle='--')
axes[0].legend(loc='best')

df_nubosidad_tarde_desconocida = df[(df['nubosidad_tarde'].isna())]
total_dias_lluvia_nubosidad_tarde_desc = sum(df_nubosidad_tarde_desconocida.llovieron_hamburguesas_al_dia_siguiente == 'si')
total_dias_sin_lluvia_nubosidad_tarde_desc = sum(df_nubosidad_tarde_desconocida.llovieron_hamburguesas_al_dia_siguiente == 'no')

sns.countplot(data=df_nubosidad_tarde_desconocida, x='llovieron_hamburguesas_al_dia_siguiente', hue='llovieron_hamburguesas_al_dia_siguiente',ax=axes[1])
axes[1].set_title("Cantidad de dias que llovieron hamburguesas al día \n siguiente para nubosidad tarde desconocida")
axes[1].set_xlabel("Llovieron hamburguesas al día siguiente")
axes[1].axhline(y=total_dias_lluvia_nubosidad_tarde_desc, color="C1", label=f"{total_dias_lluvia_nubosidad_tarde_desc} dias",  linestyle='--')
axes[1].axhline(y=total_dias_sin_lluvia_nubosidad_tarde_desc, color="C0", label=f"{total_dias_sin_lluvia_nubosidad_tarde_desc} dias", linestyle='--')
axes[1].legend(loc='best')

plt.show()

Analogamente al caso de las horas de sol desconocidas, cuando la nubosidad es desconocida (practicamente idéntico para ambas), en el 80% de los casos no llovieron hamburguesas al dia siguiente.

### 7. ¿Las direcciones del viento temprano y tarde, estarán involucradas en la búsqueda del baseline?

In [ ]:
lluvurguesas_viento_temprano = df.groupby(by="direccion_viento_temprano")['llovieron_hamburguesas_al_dia_siguiente'].value_counts(normalize=True)

lluvurguesas_viento_temprano = lluvurguesas_viento_temprano.to_frame()
lluvurguesas_viento_temprano.rename(columns={'llovieron_hamburguesas_al_dia_siguiente':'porcentaje_viento_temprano'},inplace=True)
lluvurguesas_viento_temprano.reset_index(inplace=True)

g = sns.catplot(
    data=lluvurguesas_viento_temprano, kind="bar",
    x="direccion_viento_temprano", y="porcentaje_viento_temprano", hue="llovieron_hamburguesas_al_dia_siguiente", order=df["direccion_viento_temprano"].value_counts().index,
    ci=None, height=8,aspect=2
)
plt.title("Porcentaje de lluvia de hamburguesas según dirección del viento temprano", fontsize=18)
g.despine(left=True)
plt.ylim(0,1)
plt.xticks(rotation=90)
g.set_axis_labels("Dirección del viento temprano", "Porcentaje de lluvia de hamburguesas",  fontsize=15)
plt.show()

No parece que la dirección del viento temprano tenga una influencia tajante en el comportamiento del target. Vemos que todas las direcciones tienen la mayoría rotunda de no llover hamburguesas al dia siguiente. Probemos ahora qué sucede con la dirección del viento a la tarde.

In [ ]:
lluvurguesas_viento_tarde = df.groupby(by="direccion_viento_tarde")['llovieron_hamburguesas_al_dia_siguiente'].value_counts(normalize=True)

lluvurguesas_viento_tarde = lluvurguesas_viento_tarde.to_frame()
lluvurguesas_viento_tarde.rename(columns={'llovieron_hamburguesas_al_dia_siguiente':'porcentaje_viento_tarde'},inplace=True)
lluvurguesas_viento_tarde.reset_index(inplace=True)

g = sns.catplot(
    data=lluvurguesas_viento_tarde, kind="bar",
    x="direccion_viento_tarde", y="porcentaje_viento_tarde", hue="llovieron_hamburguesas_al_dia_siguiente", order=df["direccion_viento_tarde"].value_counts().index,
    ci=None, height=8,aspect=2
)
plt.title("Porcentaje de lluvia de hamburguesas según dirección del viento tardío", fontsize=18)
g.despine(left=True)
plt.ylim(0,1)
plt.xticks(rotation=90)
g.set_axis_labels("Dirección del viento tardío", "Porcentaje de lluvia de hamburguesas",  fontsize=15)
plt.show()

Al igual que con el viento temprano, el viento a la tarde no parece tampoco influenciar el target de manera determinante como para volverlo parte de nuestra predicción, ya que si bien hay leves diferencias entre cuantas veces el target toma valores positivos y negativos para cada dirección, estas no son suficientemente sustanciales.

### 8. ¿Cómo afectan las velocidades de los vientos tempranos y tardes al baseline?

In [ ]:
fig = plt.figure( figsize=[6.4 * 2, 4.8], dpi=100)

df_llovio_h_maniana = df[df.llovieron_hamburguesas_al_dia_siguiente == 'si']
df_no_llovio_h_maniana = df[df.llovieron_hamburguesas_al_dia_siguiente == 'no']
rango_velocidad_viento_temp = [0,70]

plt.hist(df_no_llovio_h_maniana.velocidad_viendo_temprano,range=rango_velocidad_viento_temp, label='No llovieron hamburguesas al dia siguiente')
plt.hist(df_llovio_h_maniana.velocidad_viendo_temprano, alpha=0.75, range=rango_velocidad_viento_temp, label='Llovieron hamburguesas al dia siguiente')
plt.title('Velocidad viento temprano - llovieron hamburguesas al dia siguiente')
plt.ylabel("Cantidad de mediciones")
plt.xlabel("Velocidad del viento [km/h]")
plt.yscale('log')
plt.legend()
plt.show()

In [ ]:
fig = plt.figure( figsize=[6.4 * 2, 4.8], dpi=100)

df_llovio_h_maniana = df[df.llovieron_hamburguesas_al_dia_siguiente == 'si']
df_no_llovio_h_maniana = df[df.llovieron_hamburguesas_al_dia_siguiente == 'no']
rango_velocidad_viento_tarde = [0,70]

plt.hist(df_no_llovio_h_maniana.velocidad_viendo_tarde,range=rango_velocidad_viento_temp, label='No llovieron hamburguesas al dia siguiente')
plt.hist(df_llovio_h_maniana.velocidad_viendo_tarde, alpha=0.75, range=rango_velocidad_viento_temp, label='Llovieron hamburguesas al dia siguiente')
plt.title('Velocidad viento tardío - hamburguesas al dia siguiente')
plt.ylabel('Cantidad de mediciones')
plt.xlabel('Velocidad del viento[km/h]')
plt.yscale('log')
plt.legend()
plt.show()

Concluimos que la distribución del viento temprano, así como la del viento tardío, son prácticamente idénticas (salvando la diferencia de cantidad de mediciones para cada una, producto del desbalance de la proporción del target en el dataset original) lluevan hamburguesas al día siguiente o no. Esto se manifiesta en que para ambos valores de target, la proporción de registros que de distribuye en cada bin es la misma. 
Creemos importante mencionar que en el primer gráfico vemos en el último bin (60-70km/h) que aumenta la relacion porcentual de la feature target positiva con respecto a la negativa, sin embargo dado que la cantidad de casos esta en el orden de las decenas, tenerlos en cuenta para el baseline causaría overfitting. Si las mediciones donde sucediera esto fueran más, sería claramente una condición a agregar.

### 9. ¿En cuánto al viento, las ráfagas tienen efecto en el target?

In [ ]:
lluvurguesas_dir_rafaga = df.groupby(by="rafaga_viento_max_direccion")['llovieron_hamburguesas_al_dia_siguiente'].value_counts(normalize=True)

lluvurguesas_dir_rafaga = lluvurguesas_dir_rafaga.to_frame()
lluvurguesas_dir_rafaga.rename(columns={'llovieron_hamburguesas_al_dia_siguiente':'porcentaje_dir_rafaga'},inplace=True)
lluvurguesas_dir_rafaga.reset_index(inplace=True)

g = sns.catplot(
    data=lluvurguesas_dir_rafaga, kind="bar",
    x="rafaga_viento_max_direccion", y="porcentaje_dir_rafaga", hue="llovieron_hamburguesas_al_dia_siguiente", order=df["rafaga_viento_max_direccion"].value_counts().index,
    ci=None, height=8,aspect=2
)
plt.title("Porcentaje de lluvia de hamburguesas al dia siguiente \n según la dirección de la ráfaga de viento", fontsize=18)
g.despine(left=True)
plt.ylim(0,1)
plt.xticks(rotation=90)
g.set_axis_labels("Dirección de la ráfaga de máximo viento", "Porcentaje de lluvia de hamburguesas al dia siguiente",  fontsize=15)
plt.show()

Vemos que al igual que el propio viento, la dirección de su máxima ráfaga no parece tener un distribución divisiva en cuanto al target. Esto lo notamos al observar que sin importar la dirección que tomemos, se mantiene una proporción similar para los dos valores que puede tomar el target (aproximadamente 80% son 'no' y 20% 'si'). Consideraremos que esta feature no influye en el target.

¿Y qué hay de la velocidad de la ráfaga? Vemos a continuación:

In [ ]:
fig, axes = plt.subplots(ncols=1, nrows=2, figsize=[6.4 * 2, 2 * 5], dpi=100)

df_llovio_h_maniana = df[df.llovieron_hamburguesas_al_dia_siguiente == 'si']
df_no_llovio_h_maniana = df[df.llovieron_hamburguesas_al_dia_siguiente == 'no']
rango_velocidad_rafaga = [0,120]

axes[0].set_title('Velocidad máxima ráfaga de viento - hamburguesas al dia siguiente')
axes[0].hist(df_no_llovio_h_maniana.rafaga_viento_max_velocidad, range=rango_velocidad_rafaga, label='No llovieron hamburguesas al dia siguiente')
axes[0].hist(df_llovio_h_maniana.rafaga_viento_max_velocidad, alpha=0.75, range=rango_velocidad_rafaga, label='Llovieron hamburguesas al dia siguiente')
axes[0].set_ylabel('Cantidad de mediciones')
axes[0].set_xlabel('Velocidad del viento[km/h]')
#axes[0].set_yscale('log')


D1 = sns.kdeplot(
    df_no_llovio_h_maniana.rafaga_viento_max_velocidad, 
    label="No llovieron hamburguesas al dia siguiente",
    legend=True,
    ax=axes[1]
)
D1.legend()
D2 = sns.kdeplot(
    df_llovio_h_maniana.rafaga_viento_max_velocidad, label="Llovieron hamburguesas al dia siguiente", 
    legend=True,
    ax=axes[1]
)
D2.legend()
plt.grid()
plt.show()

Vemos que la velocidad de dicha ráfaga tiene también una distribución similar tanto para los casos en los que llueve al dia siguiente como en los que no, ya que si bien en el histograma se ve una gran diferencia en altura por el desbalanceo de registros segun el valor del target, el density plot no hace diferencia segun la cantidad de mediciones de cada valor. 

En la curva de los casos en los que llovieron hamburguesas al dia siguiente se observa ligeramente mejor distribuida la densidad de mediciones, agrupando la mayoria de casos entre 30 y 70km/h. Por el contrario la curva de los casos en los que no llovieron hamburguesas al dia siguiente se distribuye en su mayoría entre los valores de  30 y 50km/h. No consideramos que la diferencia sea lo suficientemente marcada como para agregar una condición sobre esta feature en el baseline.

Concluimos finalmente que ninguno de los datos respecto al viento nos da información que vayamos a utilizar para discriminar en el baseline.

 ### 10. ¿Los máximos y mínimos de temperatura, indican algo sobre si llueven o no hamburguesas al día siguiente?

In [ ]:
plt.figure(dpi=120)
plt.title("Distribución de la temperatura máxima según \n si llovieron hamburguesas o no al día siguiente")
sns.violinplot(
    data=df,
    y='temp_max',
    x='llovieron_hamburguesas_al_dia_siguiente'
#    palette=['#D17049', "#89D15E"],
)
plt.xlabel("Llovieron hamburguesas al día siguiente")
plt.ylabel("Temperatura máxima  [Celcius]")
plt.axhline(y=35, color="#382119", label="35", linestyle="dashed")
plt.axhline(y=25, color="#E0DCD3", label="25", linestyle="dashed")
plt.axhline(y=15, color="#ABC3C9", label="15", linestyle="dashed")
plt.legend(title="Grados")
plt.xticks([False, True], ["No", "Sí"])
plt.show()

In [ ]:
plt.figure(dpi=120)
plt.title("Distribución de la temperatura mínima según \n si llovieron hamburguesas o no al día siguiente")
sns.violinplot(
    data=df,
    y='temp_min',
    x='llovieron_hamburguesas_al_dia_siguiente'
)
plt.xlabel("Llovieron hamburguesas al día siguiente")
plt.ylabel("Temperatura mínima  [Celcius]")
plt.axhline(y=20, color="#382119", label="20", linestyle="dashed")
plt.axhline(y=5, color="#ABC3C9", label="5", linestyle="dashed")
plt.legend(title="Grados")
plt.xticks([False, True], ["No", "Sí"])
plt.show()

Para el caso de la temperatura máxima, observamos que la mayoría de casos donde el target es positivo, el valor de esta feature se agrupa entre 15 y 25 grados, con una gran parte del resto de sus mediciones tomando valores entre 25 y 35. Análogamente, cuando el target es negativo, esta feature también se distribuye para la mayor parte de las mediciones entre los valores 15 y 35, con la única diferencia que lo hace mas equitativamente, y no muestra el pico entre los valores 15 y 25 que se observa cuando llovieron hamburguesas al  día siguiente.

Analizando la temperatura mínima, vemos aun menos diferencias entre las distribuciones gráficadas para cada valor del target, ya que tanto para cuando es positivo, como para cuando es negativo, la cantidad más grande de mediciones se agrupan en valores entre 5 y 20 grados. La única diferencia observable a simple vista es que cuando el target es negativo, la distribución es mas suave que cuando es positivo, ya que la última presenta una forma escalonada cuando alcanza temperaturas mas elevadas que 15 grados.

Si bien vemos ligeras diferencias para ambas features, no son suficientemente grandes como para ser tenidas en cuenta a la hora de realizar la predicción, y por ende no lo serán.

### 11. ¿Podemos encontrar alguna correlación entre la máxima temperatura en el día, y los milímetros que se evaporan de lluvia?
Intuitivamente creemos que debería haber una correlación entre estas features, ya que a mayor temperatura el agua debería pasar antes a estado gaseoso. Intentemos visualizar su relación mediante un line-plot:

In [ ]:
df_mm = df.sort_values(by=['mm_evaporados_agua', 'temp_max'], ascending = False)
plt.figure(dpi=150)
sns.lineplot(
    data=df, x='temp_max', y='mm_evaporados_agua'
)
plt.show()
print("Hay " + str(len(df_mm[df_mm['mm_evaporados_agua'] > 25])) + " mediciones que contaron con más de 25mm de agua evaporados.") 


Podemos observar que los milímetros evaporados parecen escalar de forma cuasi lineal con la temperatura máxima en el día. Sin embargo, notamos que hay un gran pico de milímetros evaporados cuando la temperatura ronda los 45 grados. Podemos asociar esto a una potencial época de sequía, pero viendo que la cantidad registros que lo cumplen son menos del 1%, entonces también podría ser un caso de ruido.

### 12. ¿Tiene relación la humedad con la variable target?
Para responder esta pregunta análitica, planteamos dos pares de boxplots, uno para cada momento del día

In [ ]:
plt.figure(dpi=120)
plt.title("Distribucion de la humedad según si llovio al día siguiente o no (temprano) ")
sns.boxplot(
    data=df,
    y='humedad_temprano',
    x='llovieron_hamburguesas_al_dia_siguiente'
)
plt.ylabel("Humedad porcentual")
plt.xlabel("Llovieron hamburguesas al dia siguiente")
plt.axhline(y=80, color="blue", label="%80", alpha=0.7)
plt.legend(title="Porcentaje de humedad")
plt.show()

No es observable una gran diferencia en cuanto a las distribuciones de humedad temprana. Se ve como dato notable que la mediana de humedad para cuando llueven hamburguesas al día siguiente esta donde termina el rango intercuartilico de la humedad para los dias que no llovieron hamburguesas, sin embargo no es un dato tan fuerte como para suponer afirmaciones en la baseline. Analizaremos la humedad a la tarde para ver si podemos concluir afirmaciones más precisas.

In [ ]:
plt.figure(dpi=120)
plt.title("Distribución de la humedad según si llovio al día siguiente o no (tarde)")
sns.boxplot(
    data=df,
    y='humedad_tarde',
    x='llovieron_hamburguesas_al_dia_siguiente'
)
plt.ylabel("Humedad porcentual")
plt.xlabel("Llovieron hamburguesas al dia siguiente")
plt.axhline(y=60, color="blue", label="%60", alpha=0.7)
plt.axhline(y=84, color="red", label="%84", alpha=0.7)
plt.legend(title="Porcentaje de humedad")
plt.xticks([False, True], ["No", "Sí"])
plt.show()

En este caso se ve una diferencia mucho más clara, ya que donde termina el rango intercuartilico de los casos en los que no llueven hamburguesas al día siguiente, 
esta próximo al inicio del rango intercuartilico de los casos en los que si llueven al día siguiente. Entonces podemos suponer que si la humedad a la tarde es aproximadamente mayor a 80%, es probable que al día siguiente lluevan hamburguesas.

### 13. ¿Encontraremos alguna correlacion entre humedad, temperatura y el target, tal cual indicaría el sentido común?
En el clima de nuestra región es intuitivo pensar que para temperaturas y humedades altas, aumenta la probabilidad de que llueva, sin embargo el target no se trata de una lluvia común y corriente si no de una lluvia de hamburguesas y por ende no podemos confiar en la lógica y el sentido común. Debemos realizar un análisis mas riguroso, para lo cual planteamos un scatter plot que relaciona la humedad en el eje X, la temperatura en el eje Y, y la variable target en el color de los puntos, uno para cada momento del día (temprano y tarde).

In [ ]:
fig, ax = plt.subplots(dpi=150)

df_copia = df.copy()
df_copia['llovieron_hamburguesas_al_dia_siguiente'].replace(['no','si'],[0,1],inplace=True)
df_copia = df_copia.dropna(subset = ['llovieron_hamburguesas_al_dia_siguiente'])

sns.scatterplot(
    x='humedad_temprano',
    y='temperatura_temprano',
    hue=df_copia['llovieron_hamburguesas_al_dia_siguiente'].tolist(),
    data=df_copia,
    alpha=0.7,
)
ax.legend(title="Llovieron hamburguesas al dia siguiente")
plt.xlabel("Humedad")
plt.ylabel("Temperatura")
plt.title('Relación humedad - temperatura (temprano)')
plt.show()

In [ ]:
fig, ax = plt.subplots(dpi=150)

df_copia = df.copy()
df_copia['llovieron_hamburguesas_al_dia_siguiente'].replace(['no','si'],[0,1],inplace=True)
df_copia = df_copia.dropna(subset = ['llovieron_hamburguesas_al_dia_siguiente'])

sns.scatterplot(
    x='humedad_tarde',
    y='temperatura_tarde',
    hue=df_copia['llovieron_hamburguesas_al_dia_siguiente'].tolist(),
    data=df_copia,
    alpha=0.7,
)
ax.legend(title="Llovieron hamburguesas al dia siguiente")
plt.xlabel("Humedad")
plt.ylabel("Temperatura")
plt.title('Relación humedad - temperatura (tarde)')
plt.show()

Para ámbos momentos del día podemos observar de manera difusa y poco clara una relación lineal decreciente entre temperatura y humedad. 

En el gráfico del momento del día temprano, la mayoría de puntos indican que al día siguiente no llovieron hamburguesas. Solo para humedades muy altas (más de 80%) y temperaturas de entre 20 y 30 grados celsius vemos una leve predominancia de lluvia de hamburguesas al día siguiente. También vemos casos separados del grupo mencionado para todas las humedades, así que el primer gráfico no nos parece lo suficientemente conclusivo.

En cambio, para el gráfico del momento del día tarde, vemos una separación mas clara de los puntos en cuanto al target se refiere, siendo que la mayoría de casos donde el target es positivo se agrupan para humedades altas, de entre 70-100%, y la mayoría de casos donde dicha feature es negativa, se agrupan para valores mucho menores de humedad. 

Esto tiene sentido ya que la tarde esta mucho mas cercana del día siguiente que la mañana, es decir, si a la mañana hay una humedad alta, esta tiene mas posibilidades de cambiar antes del día siguiente que la humedad de la tarde, dejando de afectar al target.

### 14. ¿Es observable una correlación entre la presión atmosférica y el target?

In [ ]:
plt.figure(dpi=120)
plt.title("Distribución de la presión atmosférica temprana \n si llovieron hamburguesas o no al día siguiente")
sns.violinplot(
    data=df,
    y='presion_atmosferica_temprano',
    x='llovieron_hamburguesas_al_dia_siguiente'
)
plt.xlabel("Llovieron hamburguesas al día siguiente")
plt.ylabel("Presión atmosférica [hPa]")
plt.xticks([False, True], ["No", "Sí"])
plt.show()

In [ ]:
plt.figure(dpi=120)
plt.title("Distribución de la presión atmosférica a la tarde \n si llovieron hamburguesas o no al día siguiente")

df_aux = df.explode('presion_atmosferica_tarde')
df_aux['presion_atmosferica_tarde'] = df_aux['presion_atmosferica_tarde'].astype('float')

sns.violinplot(
    data=df_aux,
    y='presion_atmosferica_tarde',
    x='llovieron_hamburguesas_al_dia_siguiente'
)
plt.xlabel("Llovieron hamburguesas al día siguiente")
plt.ylabel("Presión atmosférica [hPa]")
plt.xticks([False, True], ["No", "Sí"])
plt.show()

Podemos ver que la feature de presíon atmosférica para ámbos momentos del día no muestra una fuerte correlación con la lluvia de hamburguesas al día siguiente, y esto se debe a que tanto para el momento temprano y el momento tarde, para los valores de 'si' y 'no' del target la mayoría de mediciones se agrupa entre los valores 1000 y 1030 hPa de una manera equitativa. La diferencia que se observa es un mínimo desplazamiento de unos pocos hPa hacia abajo cuando el target es positivo, pero es tan pequeño el desplazamiento que no sería correcto considerarlo como una diferencia significante.

Obtenemos como conclusión que esta feature no influye en el target y no será tenida en cuenta a la hora de predecirlo.

### 15. ¿Qué correlaciones encontramos entre horas_de_sol, nubosidad_tarde y el target?

Los features horas_de_sol y nubosidad_tarde fueron de interés durante el proceso de visualización, a pesar de tener un gran número de missings. Debido a esto puede ser útil visualizar la relación entre sí y a su vez con el target llovieron_hamburguesas_al_dia_siguiente. Además observando la nubosidad, nos dimos cuenta que solo consta de números enteros del 0 al 9, por lo que podríamos considerar que ya fue discretizada y es posible usarla en un heatmap.

Para poder compararla con las horas de sol, primero categorizaremos estas mismas según rangos:

In [ ]:
def categorizar_horas_sol(horas_de_sol):
    if horas_de_sol < 2:
        return 'Menos de 2'
    elif horas_de_sol < 4:
        return 'Entre 2 y 4'
    elif horas_de_sol < 6:
        return 'Entre 4 y 6'
    elif horas_de_sol < 8:
        return 'Entre 6 y 8'
    elif horas_de_sol < 10:
        return 'Entre 8 y 10'
    elif horas_de_sol < 12:
        return 'Entre 10 y 12'
    elif horas_de_sol < 14:
        return 'Entre 12 y 14'
    return 'Más de 14'

In [ ]:
df_copia = df.copy()
df_copia['rango_horas_sol'] = df_copia['horas_de_sol'].apply(categorizar_horas_sol)
df_copia['llovieron_hamburguesas_al_dia_siguiente'] = df_copia['llovieron_hamburguesas_al_dia_siguiente'].map({'si': 1, 'no': 0})

correlacion_sol_nubes = pd.pivot_table(data=df_copia, index = 'rango_horas_sol',values = 'llovieron_hamburguesas_al_dia_siguiente', columns = 'nubosidad_tarde')
correlacion_sol_nubes.sort_index(ascending =  False, inplace = True)
correlacion_sol_nubes = correlacion_sol_nubes.reindex(['Más de 14', 'Entre 12 y 14','Entre 10 y 12','Entre 8 y 10','Entre 6 y 8','Entre 4 y 6','Entre 2 y 4','Menos de 2'], axis=0)
display(correlacion_sol_nubes)
df_copia[df_copia.nubosidad_tarde > 8]

Al observar la matriz de correlaciones vemos que la columna de nubosidad_tarde de valor 9 tiene la mayoría de valores en nulo, esto se debe que solo hay un registro en el dataset con valor de nubosidad mayor a 8 y no será tenido en cuenta a la hora de graficar el heatmap.

In [ ]:
fig = plt.figure(dpi=150)

df_copia = df_copia[df_copia.nubosidad_tarde <= 8]
correlacion_sol_nubes_2 = pd.pivot_table(data=df_copia, index = 'rango_horas_sol',values = 'llovieron_hamburguesas_al_dia_siguiente', columns = 'nubosidad_tarde')
correlacion_sol_nubes_2.sort_index(ascending =  False, inplace = True)
correlacion_sol_nubes_2 = correlacion_sol_nubes_2.reindex(['Más de 14', 'Entre 12 y 14','Entre 10 y 12','Entre 8 y 10','Entre 6 y 8','Entre 4 y 6','Entre 2 y 4','Menos de 2'], axis=0)

mascara = correlacion_sol_nubes_2.isnull()

cmap="YlGnBu"
g = sns.heatmap(correlacion_sol_nubes_2,  annot = True, annot_kws={"fontsize":7},cmap="YlGnBu" )
plt.xlabel("Nubosidad tarde")
plt.ylabel("Horas de Sol")
plt.title('Relación nubosidad tarde - horas de sol, y llovieron hamburguesas al dia siguiente')
plt.show()

Podemos inmediatamente ver que cuando hay alta nubosidad a la tarde y pocas horas de sol en el día, se presentan condiciones favorables para que lluevan hamburguesas al día siguiente. Los gradientes en el heatmap nos dan a entender que estas features están inversamente correlacionadas, y podremos tomar provecho de esto una vez planteadas nuestras conclusiones. Sin embargo hay algunas excepciones, que nos dan a entender que la nubosidad esta más correlacionada con el target que horas de sol, y estos son los casos donde nubosidad toma valores de 6 en adelante, y horas de sol toma valores mayores a 14. En estas 3 secciones se rompe la relación de proporcionalidad inversa ya que el porcentaje de target positivo toma valores más elevados, similares a cuando las horas de sol toman valores entre 2 y 4, y por ende se termina observando algo más similar a una proporcionalidad directa.

### 16. ¿Cómo se comporta el target cuando las features que prometen mejor predicción están todas en valores nulos?

Queremos ver qué sucede con el target cuando simultáneamente se encuentran todas nuestras variables candidatas del baseline en un estado nulo.

In [ ]:
plt.figure(figsize=(5.4, 3.8), dpi=150)

df_triple_missing = df[(df['horas_de_sol'].isna() & df['nubosidad_tarde'].isna() & df['humedad_tarde'].isna())]

labels = ['No', 'Si']
sizes = df['llovieron_hamburguesas_al_dia_siguiente'].value_counts()
explode = (0, 0.1)
plt.title('Llueven hamburguesas al dia siguiente')
plt.pie(sizes, explode=explode, data = df['llovieron_hamburguesas_al_dia_siguiente'],labels = labels , autopct='%1.1f%%',
        shadow=True, startangle=90)
plt.show()


plt.figure(figsize=(5.4, 3.8), dpi=150)
labels = ['No', 'Si']
sizes = df_triple_missing['llovieron_hamburguesas_al_dia_siguiente'].value_counts()
explode = (0, 0.1)
plt.title('Llueven hamburguesas al dia siguiente cuando \n las tres features de interés están missing')
plt.pie(sizes, explode=explode, data = df_triple_missing['llovieron_hamburguesas_al_dia_siguiente'],labels = labels , autopct='%1.1f%%',
        shadow=True, startangle=90)

plt.show()

Podemos ver que hay una diferencia porcentual de alrededor del 5% a favor del valor 'Si' cuando dichas tres features están en missing, si lo comparamos con el dataframe entero. Debido a esto, tomaremos la decisión de no droppear dichos registros a la hora de formar una baseline.

### 17. ¿Cómo afecta al target la cantidad de features nulas en una medición?

In [ ]:
df_copia = df.copy()
df_copia['cantidad_features_null'] = df_copia.isnull().sum(axis=1)

lluvurguesas_features_null = df_copia.groupby(by="cantidad_features_null")['llovieron_hamburguesas_al_dia_siguiente'].value_counts(normalize=True)
lluvurguesas_features_null = lluvurguesas_features_null.to_frame()
lluvurguesas_features_null.rename(columns={'llovieron_hamburguesas_al_dia_siguiente':'porcentaje_feature_missing'},inplace=True)
lluvurguesas_features_null.reset_index(inplace=True)
g = sns.catplot(
    data=lluvurguesas_features_null, kind="bar",
    x="cantidad_features_null", y="porcentaje_feature_missing", hue="llovieron_hamburguesas_al_dia_siguiente",
    ci=None, height=8,aspect=2
)

plt.title("Porcentaje de lluvia de hamburguesas al dia siguiente \n según la cantidad de missings en mediciones", fontsize=18)
g.despine(left=True)
plt.ylim(0,1)
plt.xticks(rotation=90)
g.set_axis_labels("Cantidad de missings en registro", "Porcentaje de lluvia de hamburguesas al dia siguiente",  fontsize=15)
plt.show()

Podemos detectar una anomalía cuando hay exactamente 13 valores nulos en las mediciones. Cuando esto sucede, el target se encuentra predominantemente en el valor "Sí" de una forma desproporcionada, y contradiciendo a la media del dataset. Esto podría tratarse de ruido en mediciones que fueron de poca calidad, así que observamos en cuántos registros pasa esto para determinar si los excluiremos del dataset:

In [ ]:
cantidad_13_nulos = len(df_copia[df_copia['cantidad_features_null'] == 13])
porcentaje_13_nulos = (len(df_copia[df_copia['cantidad_features_null'] == 13]) / len(df_copia)) * 100
print("La cantidad de mediciones con 13 features en missing fue: " + str(cantidad_13_nulos))
print("Del total, esto es un : " + str(round(porcentaje_13_nulos, 4)) + "%")

Al haber sido un porcentaje tan bajo del total, podemos con confianza tomar el criterio de eliminar estos registros sospechosos antes de confeccionar la baseline.

### 18. ¿Son de utilidad las mediciones donde el target tiene el valor NaN?

Recordando como se ve a continuación, que el porcentaje de missings en la variable target es relativamente bajo, es válido plantearnos si seria conveniente droppear del dataset los registros que tengan un NaN en la variable target.

In [ ]:
df_con_nulos = df_nulos[df_nulos['Cantidad de nulos'] > 0]
df_con_nulos.sort_values(by=['Porcentaje de nulos']).head()

Como podemos ver, llovieron_hamburguesas_al_dia_siguiente tiene missings en un ~2.3% de los registros del dataset, por lo que no sería a primera vista muy grave droppear aquellas mediciones con el target en NaN. Sin embargo, también es de importancia tener en cuenta que a la hora de predecir el target en nuestro algoritmo no tendremos con qué comparar la estimación de dicho algoritmo si la feature pertinente se encuentra en NaN. Esto nos resulta un importante indicativo de que terminaremos ignorando en el baseline las mediciones con missings en el target. 

Aun así, vamos a comparar para estar seguros las distribuciones de las feature humedad_tarde, horas_sol y nubosidad_tarde cuando el target está en NaN contra cuando no. Ya que estas features son las que nosotros consideramos con mejor potencial de predicción, si no tienen un cambio radical de comportamiento cuando el target tiene missings, entonces procederemos a droppear dichos registros.

In [ ]:
fig = plt.figure(figsize=[6.4 * 2, 4.8], dpi=100)

df_target_missings = df[(df['llovieron_hamburguesas_al_dia_siguiente'].isna())]
df_target_completo = df[(df['llovieron_hamburguesas_al_dia_siguiente'].isna() == False)] 

plt.hist(df_target_completo.humedad_tarde, label='Humedad cuando el target tiene valor')
plt.hist(df_target_missings.humedad_tarde, alpha=0.75, label='Humedad cuando el target es nulo')
plt.yscale("log")

plt.ylabel("Cantidad")
plt.xlabel("Humedad")

plt.legend()
plt.show()

Vemos que a pesar de ser una cantidad mucho menor de registros (lo cuál es esperado habiendo solo un 2.3% de missings en el target), se mantiene la forma de la distribución de humedad a pesar de no conocer el valor de llovieron_hamburguesas_al_dia_siguiente. Comprobemos qué sucede con las horas de sol:

In [ ]:
fig = plt.figure(figsize=[6.4 * 2, 4.8], dpi=100)

plt.hist(df_target_completo.horas_de_sol, label='Horas de sol cuando el target tiene valor')
plt.hist(df_target_missings.horas_de_sol, alpha=0.75, label='Horas de sol cuando el target es nulo')
plt.yscale("log")

plt.ylabel("Cantidad")
plt.xlabel("Horas de sol")

plt.legend()
plt.show()

En el caso de las horas de sol, tenemos una situación similar donde la forma de la distribución se mantiene independientemente de los missings del target, si bien obviamente la escala sigue siendo muy distinta. Pasemos a analizar la nubosidad:

In [ ]:
fig = plt.figure(figsize=[6.4 * 2, 4.8], dpi=100)

plt.hist(df_target_completo.nubosidad_tarde,range=[0,9], label='Nubosidad a la tarde cuando el target tiene valor')
plt.hist(df_target_missings.nubosidad_tarde,range=[0,9], alpha=0.75, label='Nubosidad a la tarde cuando el target es nulo')
plt.yscale("log")

plt.ylabel("Cantidad")
plt.xlabel("Nubosidad a la tarde")

plt.legend()
plt.show()

En este caso la diferencia de escalas es mucho más grosera, pues podemos ver que para nubosidades muy bajas tenemos una cantidad de registros que ronda los dígitos dobles si el target es nulo, mientras que cuando no es nulo tenemos alrededor de 4000 registros con dicho valor de nubosidad.

A pesar de esto, sigue sin ser una visualización que nos indique una correlación entre la existencia de un valor concreto del target y la nubosidad, por lo que nuevamente se refuerza nuestra hipótesis, e ignoraremos del dataset aquellos registros donde el target sea nulo.

# Conclusiones de la visualizacion y análisis:
 * La humedad de la tarde parece ser una buena indicadora de la variable target, siendo así que será considerada como feature principal de predicción dentro de nuestra baseline.
 * Las features horas de sol y de nubosidades, a pesar de tener una gran cantidad de missings, fueron destacadas ya que también discriminan de manera clara los casos en los que llueven hamburguesas al día siguiente de los que no.
 * La feature "llovieron_hamburguesas_hoy" también sera considerada para predecir el target ya que cuando su valor es 'si', la probabilidad de que llovieron_hamburguesas_al_dia_siguiente sea positiva aumenta drásticamente con respecto a cuando no.
 * Hay una gran cantidad de features de las cuales no pudimos sacar conclusiones que nos ayuden a predecir el valor de la variable target ya que poseian una distribución equitativa en todos sus valores posibles, para ambos valores del target.

# Baselines posibles
### Comencemos por definir una serie de baselines posibles según nuestro análisis previo, con el objetivo de intentar predecir la variable target.

#### Para esto primero crearemos una función de preprocesamiento, la cuál se encargará de:
    1. Droppear del dataset los registros que tengan un NaN en la variable target, por lo cual no podemos saber si nuestra predicción está bien o no, ni aprender nada de ellos.
    2. Droppear del dataset los registros que tengan exactamente 13 de campos en NaN, los cuales consideramos mediciones de poca calidad e incompletas. Para esto, tomamos como treshold tener 12 no nulos. 

In [ ]:
def preprocesar_df(df_raw:pd.DataFrame):
    df_raw = df_raw.dropna(subset=['llovieron_hamburguesas_al_dia_siguiente'])
    df_raw = df_raw.dropna(thresh = len(df_raw.columns) - 12)
    return df_raw

#### Una vez definida la función de preprocesamiento, programamos una serie de diferentes baselines que intuimos superaran el 80% de accuracy al predecir el target:

In [ ]:
def baseline(df_baseline:pd.DataFrame):
    
    llovio_h_hoy = (df_baseline['llovieron_hamburguesas_hoy'] == 'si')
    
    alta_humedad_tarde = (df_baseline['humedad_tarde'] >= 79)
    horas_sol_bajas = (df_baseline['horas_de_sol'] <= 7.5)
    alta_nubosidad_tarde = (df_baseline['nubosidad_tarde'] >= 7.1)
    
    df_baseline['prediccion_llueven_h_al_dia_siguiente'] = 'no'
    df.loc[alta_humedad_tarde, 'prediccion_llueven_h_al_dia_siguiente'] = 'si'
    df.loc[llovio_h_hoy & alta_nubosidad_tarde & horas_sol_bajas, 'prediccion_llueven_h_al_dia_siguiente'] = 'si'
    
    resultado =  df_baseline['prediccion_llueven_h_al_dia_siguiente'].values.tolist()
    df_baseline.drop(columns=['prediccion_llueven_h_al_dia_siguiente']) #remuevo la columna de predicciones para no alterar el df original
    return resultado

def baseline_2(df_baseline:pd.DataFrame):
    
    alta_humedad_tarde = (df_baseline['humedad_tarde'] >= 79)
    
    df_baseline['prediccion_llueven_h_al_dia_siguiente'] = 'no'
    df.loc[alta_humedad_tarde, 'prediccion_llueven_h_al_dia_siguiente'] = 'si'
    
    resultado =  df_baseline['prediccion_llueven_h_al_dia_siguiente'].values.tolist()
    df_baseline.drop(columns=['prediccion_llueven_h_al_dia_siguiente'])
    return resultado

def baseline_3(df_baseline:pd.DataFrame):
    
    llovio_h_hoy = (df_baseline['llovieron_hamburguesas_hoy'] == 'si')
    alta_humedad_tarde = (df_baseline['humedad_tarde'] >= 79)
    alta_nubosidad_tarde = (df_baseline['nubosidad_tarde'] >= 7.1)

    df_baseline['prediccion_llueven_h_al_dia_siguiente'] = 'no'
    df.loc[alta_humedad_tarde, 'prediccion_llueven_h_al_dia_siguiente'] = 'si'
    df.loc[llovio_h_hoy & alta_nubosidad_tarde , 'prediccion_llueven_h_al_dia_siguiente'] = 'si'
    
    resultado =  df_baseline['prediccion_llueven_h_al_dia_siguiente'].values.tolist()
    df_baseline.drop(columns=['prediccion_llueven_h_al_dia_siguiente']) #remuevo la columna de predicciones para no alterar el df original
    return resultado

# Predicción según baselines
### Pasamos entonces a probar las funciones baselines definidas y a comparar las accuracies resultantes:

Para el primer baseline, que tiene en cuenta principalmente la feature humedad_tarde, y luego también predice que llueve al dia siguiente para aquellas filas donde haya alta nubosidad a la tarde, bajas horas de sol, y haya llovido hamburguesas hoy, la accuracy resulta:

In [ ]:
df = preprocesar_df(df_original)
(baseline(df) == df['llovieron_hamburguesas_al_dia_siguiente']).mean().round(4)

Para el segundo baseline, que se guia únicamente en si hubo una alta humedad a la tarde o no, la accuracy resulta:

In [ ]:
df = preprocesar_df(df_original)
(baseline_2(df) == df['llovieron_hamburguesas_al_dia_siguiente']).mean().round(4)

Finalmente nuestro tercer baseline es muy parecido al primero, con la diferencia de ignorar las horas de sol:

In [ ]:
df = preprocesar_df(df_original)
(baseline_3(df) == df['llovieron_hamburguesas_al_dia_siguiente']).mean().round(4)

### Se observa que en caso de tener que elegir entre nuestros baselines, aquel con un puntaje de accuracy más elevado, entonces se utilizaría el primero. Sin embargo consideramos de alta importancia notar que el segundo baseline es solamente un 0,1% menos preciso, siendo a su vez más simple por necesitar solamente de la feature humedad_tarde para predecir.